In [1]:

%load_ext autoreload
%autoreload 2

In [24]:

from io import BytesIO
from block import Block, FULL_GENESIS_BLOCK

genesis = Block.parse(BytesIO(FULL_GENESIS_BLOCK))

print(genesis)

In [4]:
genesis.bits

b'\xff\xff\x00\x1d'

In [5]:
genesis.version

1

In [6]:
genesis.merkle_root

b'J^\x1eK\xaa\xb8\x9f:2Q\x8a\x88\xc3\x1b\xc8\x7fa\x8fvg>,\xc7z\xb2\x12{z\xfd\xed\xa3;'

In [7]:
from helper import merkle_root

merkle_root([tx.hash() for tx in genesis.txns])

b'J^\x1eK\xaa\xb8\x9f:2Q\x8a\x88\xc3\x1b\xc8\x7fa\x8fvg>,\xc7z\xb2\x12{z\xfd\xed\xa3;'

In [8]:
genesis.merkle_root == merkle_root([tx.hash() for tx in genesis.txns])

True

In [43]:
from helper import little_endian_to_int, hash256, bits_to_target, int_to_little_endian, target_to_bits

def mine(block):
    target = bits_to_target(block.bits)
    nonce = 0
    serialized_block = block.serialize()
    nonce_index = 76
    while True:
        ser = serialized_block[:76] + int_to_little_endian(nonce, 4) + serialized_block[80:]
        proof = little_endian_to_int(hash256(ser))
        if proof < target:
            block.nonce = int_to_little_endian(nonce, 4)
            return block
        else:
            nonce += 1
        if nonce % 100_000_000 == 0:
            print(nonce)
        

In [47]:
genesis.bits = target_to_bits(16**62)
mine(genesis)